In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json




In [43]:
df = pd.read_csv(r'C:\Users\Vipul\Property-Click-Prediction\notebook\datasets\property_data_set.csv',parse_dates = ['activation_date'], dayfirst=True)

df1 = pd.read_csv(r"C:\Users\Vipul\Property-Click-Prediction\notebook\datasets\property_interactions.csv",parse_dates = ['request_date'] , dayfirst=True)

df2 = pd.read_table(r"C:\Users\Vipul\Property-Click-Prediction\notebook\datasets\property_photos.tsv")

In [44]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 10)

In [45]:
df.sample(2)

,property_id,type,activation_date,bathroom,floor,total_floor,furnishing,gym,latitude,longitude,lease_type,lift,locality,parking,property_age,property_size,swimming_pool,pin_code,rent,deposit,building_type
18311,ff8081815a833856015a8365ff83167b,BHK1,2017-02-28 12:45:00,1,0,2.0,SEMI_FURNISHED,0,12.919365,77.541982,ANYONE,0,Ittamadu,TWO_WHEELER,1,350,0,560085.0,5500,40000,IF
6994,ff808181588627250158868455b21498,BHK1,2017-03-08 11:55:00,1,1,1.0,SEMI_FURNISHED,0,12.867876,77.568272,ANYONE,0,JP Nagar,TWO_WHEELER,0,500,0,560062.0,6500,65000,IF


In [46]:
print('Property data Shape', df.shape)
print('Pics data Shape',df1.shape)
print('Interaction data Shape',df2.shape)

Property data Shape (28888, 21)
Pics data Shape (170611, 2)
Interaction data Shape (28888, 2)


In [47]:
df1.sample(3)

,property_id,request_date
133815,ff8081815aae6093015aaeaf7017132c,2017-03-10 16:40:08
84554,ff8081815a30d446015a35d18c4233e6,2017-02-14 19:17:01
5131,ff8081814db00698014db411818c1667,2017-03-26 15:26:02


In [48]:
df2.sample(2)

,property_id,photo_urls
22178,ff8081815ab73de0015ab7860e8e0d1f,"[{\title\"":\""\"",\""name\"":\""files[]\"",\""imagesM..."
20782,ff8081815aa4011f015aa71702ea781c,"[{\title\"":\""\"",\""name\"":\""files[]\"",\""imagesM..."


# HANDLING PICS DATASET

In [49]:
df2.head(5)

,property_id,photo_urls
0,ff808081469fd6e20146a5af948000ea,"[{\title\"":\""Balcony\"",\""name\"":\""IMG_20131006..."
1,ff8080814702d3d10147068359d200cd,NaN
2,ff808081470c645401470fb03f5800a6,"[{\title\"":\""Bedroom\"",\""name\"":\""ff808081470c..."
3,ff808081470c6454014715eaa5960281,"[{\title\"":\""Bedroom\"",\""name\"":\""Screenshot_7..."
4,ff808081474aa867014771a0298f0aa6,"[{\title\"":\""Other\"",\""name\"":\""ff808081474aa8..."


In [50]:
df2.shape

(28888, 2)

In [51]:
df2.dtypes

property_id    object
photo_urls     object
dtype: object

In [52]:
df2.isna().sum()

property_id        0
photo_urls     11022
dtype: int64

In [53]:
# Try to correct the first Json
text_before = df2['photo_urls'][0]
print('Before Correction: \n\n', text_before)
# Try to replace corrupted values then convert to json 
text_after = text_before.replace('\\' , '').replace('{title','{"title').replace(']"' , ']').replace('],"', ']","')
print("\n\nAfter correction and converted to json: \n\n", json.loads(text_after))

Before Correction: 

 [{\title\":\"Balcony\",\"name\":\"IMG_20131006_120837.jpg\",\"imagesMap\":{\"original\":\"ff808081469fd6e20146a5af948000ea_65149_original.jpg\",\"thumbnail\":\"ff808081469fd6e20146a5af948000ea_65149_thumbnail.jpg\",\"medium\":\"ff808081469fd6e20146a5af948000ea_65149_medium.jpg\",\"large\":\"ff808081469fd6e20146a5af948000ea_65149_large.jpg\"},\"displayPic\":false},{\"title\":\"Bathroom\",\"name\":\"IMG_20131006_120734.jpg\",\"imagesMap\":{\"original\":\"ff808081469fd6e20146a5af948000ea_63511_original.jpg\",\"thumbnail\":\"ff808081469fd6e20146a5af948000ea_63511_thumbnail.jpg\",\"medium\":\"ff808081469fd6e20146a5af948000ea_63511_medium.jpg\",\"large\":\"ff808081469fd6e20146a5af948000ea_63511_large.jpg\"},\"displayPic\":false},{\"title\":\"Bedroom\",\"name\":\"IMG_20131006_120643.jpg\",\"imagesMap\":{\"original\":\"ff808081469fd6e20146a5af948000ea_16708_original.jpg\",\"thumbnail\":\"ff808081469fd6e20146a5af948000ea_16708_thumbnail.jpg\",\"medium\":\"ff808081469fd6e20

In [54]:
# Function to correct corrupted json and get count of photos
def correction (x):
    # if value is null put count with 0 photos
    if x is np.nan or x == 'NaN':
        return 0
    else :
        # Replace corrupted values then convert to json and get count of photos
        return len(json.loads( x.replace('\\' , '').replace('{title','{"title').replace(']"' , ']').replace('],"', ']","') ))
        
# Apply Correction Function
df2['photo_count'] = df2['photo_urls'].apply(correction)

In [55]:
# Delete photo_urls column 
del df2['photo_urls']
# Sample of Pics data
df2.sample(5)

,property_id,photo_count
20907,ff8081815aa402ce015aa7be31480a0a,7
26474,ff8081815af67c48015af9a9ad4e654c,6
24236,ff8081815ad5e9a4015ad6afc0695c2c,0
9700,ff80818159fa0f0f0159fe1e5de84b1f,11
12315,ff8081815a21c1b1015a26a9901033a3,4


To make more than one model and achieve the best of them, we assign more than one label here:


1.   The number of interactions within 3 days
2.   Interactions divided into categories within 3 days
3.   The number of interactions within 7 days
4.   Interactions divided into categories within 7 days

# Number of Interaction Within 3 Days